In [1]:
import sys
from pathlib import Path

if str(Path().resolve().parent) not in sys.path:
    sys.path.append(str(Path().resolve().parent))

# Data Processing

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from src.config import random_seed, PREPROCESSED_BLOG_DATASET_PATH

# Import data
blogs = pd.read_csv(PREPROCESSED_BLOG_DATASET_PATH)

# Split dataset into validation and test set
Xval, Xtest, yval_score, ytest_score = train_test_split(
    blogs.drop(columns=['normalized_engagement_score']), blogs["normalized_engagement_score"],
    test_size=0.4, random_state=random_seed)

# Same Xval, Xtest; new explained variable "engagement_level"
Xval, Xtest, yval_level, ytest_level = train_test_split(
    blogs.drop(columns=['engagement_level', 'normalized_engagement_score']), blogs["engagement_level"],
    test_size=0.4, random_state=random_seed)

print(f"Size of validation set, X: {Xval.shape}, y: {yval_score.shape}")
print(f"Size of test set, X: {Xtest.shape}, y: {ytest_score.shape}")

Size of validation set, X: (30, 10), y: (30,)
Size of test set, X: (20, 10), y: (20,)


In [3]:
valid_blogs = blogs[blogs["engagement_level"].isin(["Good", "Very Good", "Excellent"])].copy()
valid_blogs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28 entries, 0 to 47
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           28 non-null     int64  
 1   title_blog                   28 non-null     object 
 2   url_blog                     28 non-null     object 
 3   author_blog                  28 non-null     object 
 4   author_followers             28 non-null     int64  
 5   claps                        28 non-null     int64  
 6   comments                     28 non-null     int64  
 7   title_paper                  28 non-null     object 
 8   url_paper                    28 non-null     object 
 9   engagement_score             28 non-null     float64
 10  normalized_engagement_score  28 non-null     float64
 11  engagement_level             28 non-null     object 
dtypes: float64(2), int64(4), object(6)
memory usage: 2.8+ KB


In [4]:
from src.text_extraction import *

valid_blogs["full_paper"] = valid_blogs["url_paper"].apply(extract_paper_text)

# Vector storage

In [5]:
import warnings
from langchain.vectorstores import FAISS
from src.models_setup import embedding_model, langchain_embedding_model
from src.config import VECTOR_STORE_PATH

warnings.filterwarnings("ignore", category=FutureWarning)

elements = []
for text, blog_url, author, claps, comments in zip(valid_blogs["full_paper"],
                                                   valid_blogs["url_blog"],
                                                   valid_blogs["author_blog"],
                                                   valid_blogs["claps"],
                                                   valid_blogs["comments"]):
    embedding = embedding_model.encode(text, clean_up_tokenization_spaces=True)
    metadata = {
        "full_text": text,
        "blog_url": blog_url,
        "author": author,
        "claps": claps,
        "comments": comments
    }
    elements.append((embedding, metadata))

vector_store = FAISS.from_texts(
    texts=[element[1]["full_text"] for element in elements],
    embedding=langchain_embedding_model,
    metadatas=[element[1] for element in elements]
)

vector_store.save_local(VECTOR_STORE_PATH)

2025-03-29 19:46:01,583 - INFO - Successfully loaded Google API key from environment variables.
2025-03-29 19:46:01,584 - INFO - Google API successfully configured with the API key.
2025-03-29 19:46:01,671 - INFO - Use pytorch device_name: cuda
2025-03-29 19:46:01,671 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
/home/kanstantsin-downar/virtualPython/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
2025-03-29 19:46:03,600 - INFO - Use pytorch device_name: cuda
2025-03-29 19:46:03,600 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-03-29 19:46:10,273 - INFO - Loading faiss with AVX2 support.
2025-03-29 19:46:10,323 - INFO - Successfully loaded faiss with AVX2 support.
2025-03-29 19:46:10,331 - INFO - Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes.


In [6]:
vector_store = FAISS.load_local(VECTOR_STORE_PATH,
                                embeddings=langchain_embedding_model,
                                allow_dangerous_deserialization=True)

def find_most_similar_article(query_text):
    query_embedding = embedding_model.encode(query_text, clean_up_tokenization_spaces=True)
    results = vector_store.similarity_search_by_vector(query_embedding, k=2)

    if results:
        most_similar = results[1]
        return most_similar.metadata

# Generator manual test

In [ ]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [14]:
from src.prompts import prompt_rag
from src.models_setup import gemini_2_flash
from src.output_formats import BlogGeneration

paper_text = extract_paper_text(blogs.loc[0, "url_paper"])
most_similar_article = find_most_similar_article(paper_text)
example_blog = extract_blog_text(url_blog=most_similar_article["blog_url"],
                                 author_blog=most_similar_article["author"],
                                 claps=most_similar_article["claps"],
                                 comments=most_similar_article["comments"])

# RAG approach
llm_generator = gemini_2_flash.with_structured_output(BlogGeneration, include_raw=True)
generation_chain = prompt_rag | llm_generator
generator_response = generation_chain.invoke({"paper_text": paper_text,
                                              "example_paper": most_similar_article["full_text"],
                                              "example_blog": example_blog})

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
print(generator_response["parsed"].text)

"""# The Landscape of Emerging AI Agent Architectures for Reasoning, Planning, and Tool Calling: A Survey by Tong Xiao and Jingbo Zhu: A Deep Dive into LLMs

TDS Archive
Large language models (LLMs) have revolutionized the ﬁeld of artiﬁcial intelligence, particularly in natural language processing. A recent book by Tong Xiao and Jingbo Zhu, serves as a comprehensive guide to the fundamental concepts and techniques behind these groundbreaking models.

# The Origin and Impact of Large Language Models
LLMs, originating from natural language processing, have become a revolutionary force in artiﬁcial intelligence. Their key innovation lies in acquiring world and language knowledge through large-scale language modeling tasks, creating universal models capable of addressing diverse problems. This approach has profoundly reshaped research methodologies in natural language processing and related ﬁelds, shifting from specialized systems trained from scratch to foundation models obtained through 

In [16]:
from src.helpers import get_examples
from src.output_formats import BlogClassification
from src.prompts import prompt_five_shots

examples = get_examples()
blog_text = generator_response["parsed"].text

llm_evaluator = gemini_2_flash.with_structured_output(BlogClassification, include_raw=True)
evaluation_chain = prompt_five_shots | llm_evaluator
evaluator_response = evaluation_chain.invoke({**examples, "blog_text": blog_text})

In [17]:
print(f"Overall assessment: {evaluator_response["parsed"].overall_assessment}")

Overall assessment: Average


In [18]:
improvements = evaluator_response["parsed"].improvements
print(f"Possible improvements:")
for i, improvement in enumerate(improvements):
    print(f"{i+1}. {improvement}")

Possible improvements:
1. Incorporate more real-world examples to illustrate the concepts discussed.
2. Add a section discussing the limitations or challenges of the surveyed architectures.
3. Use a more engaging and less academic writing style.
4. Include a call to action to encourage readers to share their thoughts or experiences with AI agent architectures.
5. Add a personal perspective or original analysis of the survey's findings.
6. Make the introduction more captivating to immediately draw the reader in.
7. Consider adding visuals or interactive elements to enhance engagement, though this is outside the scope of the textual analysis.
8. Shorten the title to make it more attractive and focused. For example: "AI Agent Architectures: A Survey Deep Dive"


In [19]:
from src.prompts import prompt_retry

possible_improvements = "\n".join([f"{i+1}. {improvement}" for i, improvement in enumerate(improvements)])

# Reflexion
generation_chain = prompt_retry | llm_generator
generator_response = generation_chain.invoke({"generated_blog": blog_text,
                                              "possible_improvements": possible_improvements})

In [20]:
print(generator_response["parsed"].text)

"""# AI Agent Architectures: A Survey Deep Dive

Forget science fiction—AI agents are rapidly becoming reality! A groundbreaking survey by Tong Xiao and Jingbo Zhu delves into the exciting landscape of AI agent architectures, particularly focusing on how Large Language Models (LLMs) are driving innovation in reasoning, planning, and tool use. This isn't just another academic paper; it's a roadmap to the future of AI. Let's break it down.

# The LLM Revolution: From NLP to Universal Problem Solvers

LLMs have exploded onto the scene, transforming artificial intelligence, especially in natural language processing. Xiao and Zhu's survey highlights how these models learn vast amounts of world and language knowledge through massive datasets. This allows them to tackle diverse problems, shifting the paradigm from specialized, custom-built systems to adaptable foundation models. Think of it as moving from building a specific tool for each job to creating a universal adapter that can learn to 

In [21]:
blog_text = generator_response["parsed"].text
evaluator_response = evaluation_chain.invoke({**examples, "blog_text": blog_text})

In [22]:
print(f"Overall assessment: {evaluator_response["parsed"].overall_assessment}")

Overall assessment: Good


# End-to-end generation test

In [1]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [2]:
from src.blog_generator import BlogGenerator
import logging

# Set up logging
if not logging.getLogger().hasHandlers():
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

generator = BlogGenerator()

2025-03-29 21:39:11,126 - INFO - Loading vector store...
2025-03-29 21:39:11,128 - INFO - Loading faiss with AVX2 support.
2025-03-29 21:39:11,171 - INFO - Successfully loaded faiss with AVX2 support.
2025-03-29 21:39:11,179 - INFO - Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes.
2025-03-29 21:39:11,184 - INFO - Vector store loaded successfully.
2025-03-29 21:39:11,185 - INFO - BlogGenerator initialized with vector store.


In [3]:
import pandas as pd
from src.config import PREPROCESSED_BLOG_DATASET_PATH

blogs = pd.read_csv(PREPROCESSED_BLOG_DATASET_PATH)

In [4]:
blog = generator.generate_blog(blogs.loc[0, "url_paper"])

2025-03-29 21:41:48,293 - INFO - Extracting paper text from URL: https://arxiv.org/pdf/2501.09223
2025-03-29 21:41:49,481 - INFO - Attempt number 1: Generating blog...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-03-29 21:41:50,673 - INFO - Found most similar article.
2025-03-29 21:41:59,348 - INFO - Checking request limits before invoking the model...
2025-03-29 21:41:59,349 - INFO - Invoking the model...
2025-03-29 21:42:11,345 - INFO - Model invoked successfully. Total requests today: 1, RPM: 1, TPM: 184121
2025-03-29 21:42:11,346 - INFO - Blog generated successfully.
2025-03-29 21:42:11,346 - INFO - Checking request limits before invoking the model...
2025-03-29 21:42:11,347 - INFO - Invoking the model...
2025-03-29 21:42:26,902 - INFO - Model invoked successfully. Total requests today: 2, RPM: 2, TPM: 198601
2025-03-29 21:42:26,903 - INFO - Blog evaluated successfully. Evaluation: Average.
2025-03-29 21:42:26,903 - INFO - Blog generation attempt 1 unsuccessful. Retrying...
2025-03-29 21:42:26,904 - INFO - Attempt number 2: Generating blog...
2025-03-29 21:42:26,905 - INFO - Checking request limits before invoking the model...
2025-03-29 21:42:26,905 - INFO - Invoking the model...
2025

In [5]:
print(blog)

# Unlocking the Foundations of Large Language Models: A Comprehensive Guide

**Table of Contents**

*   [Introduction to Large Language Models (LLMs)](#introduction-to-large-language-models-llms)
*   [From Specialized Systems to Foundation Models](#from-specialized-systems-to-foundation-models)
*   [Key Concepts and Techniques](#key-concepts-and-techniques)
    *   [Pre-training: The Bedrock of LLMs](#pre-training-the-bedrock-of-llms)
    *   [Generative Models: Creating Human-Like Text](#generative-models-creating-human-like-text)
    *   [Prompting: Guiding LLMs to Perform Tasks](#prompting-guiding-llms-to-perform-tasks)
    *   [Alignment: Ensuring LLMs are Helpful and Harmless](#alignment-ensuring-llms-are-helpful-and-harmless)
*   [Why This Matters](#why-this-matters)
*   [Challenges and Limitations of LLMs](#challenges-and-limitations-of-llms)
*   [Ethical Considerations](#ethical-considerations)
*   [Future Trends and Directions](#future-trends-and-directions)
*   [Dive Deeper](